In [1]:
# -*- coding: utf-8 -*-
"""
Green Space Quality – ViT-Only PyTorch Training Pipeline
========================================================
מודל ייעודי ל-ViT (Vision Transformer) בלבד, עם:
- חלוקה 70/10/20 (Train/Val/Test) – סטרטיפית
- ViT-B/16 Pretrained (ImageNet)
- הקפאת ה-encoder, אימון רק של הראש (Classifier Head)
- אוגמנטציות בסגנון המאמר (Albumentations)
- מדדים בכל Epoch על סט ה-Validation:
    Accuracy, Precision_macro, Recall_macro, F1_macro, Cohen's Kappa, ROC-AUC Macro (OVR)
- מדדים מלאים + Confusion Matrix על סט ה-Test

מתאים להרצה מ-Notebook על Windows (num_workers=0, tqdm.notebook וכו').
"""

import os
import json
import random
from pathlib import Path
from typing import List, Tuple

import numpy as np
import cv2
import albumentations as A

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    cohen_kappa_score,
)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from tqdm.notebook import tqdm

# -----------------------------
# 🔹 Config – הגדרות גלובליות
# -----------------------------

class Config:
    # נתיב לתיקיית התמונות (תיקייה אחת עם 3 תיקיות משנה: Healthy/Dried/Contaminated)
    root_dir = r"C:\afeca academy\year 2\CVDLP\GreenSpaceQualityProject\OriginalPictures"

    # ארכיטקטורה – כאן רק ViT
    arch = "vit_b_16"

    # גודל batch
    batch = 32

    # מספר Epochs מקסימלי
    epochs = 50

    # seed לרנדום
    seed = 42

    # תיקייה לשמירת המודל, ההיסטוריה והמדדים
    save_dir = "outputs_gsq_vit"

    # Early Stopping – כמה Epochs ללא שיפור לפני עצירה
    patience = 7

# "args" כעצם קונפיגורציה יחיד
args = Config()

# -----------------------------
# מחלקות ופרמטרים גנריים
# -----------------------------

CLASS_NAMES = ["Healthy", "Dried", "Contaminated"]
NUM_CLASSES = len(CLASS_NAMES)

IMG_SIZE = 224  # מתאים גם ל-ViT-B/16 של torchvision
DEFAULT_SAVE_DIR = args.save_dir

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# -----------------------------
# Dataset & Augmentations
# -----------------------------

def build_train_transform(img_size: int = 224):
    """אוגמנטציות בסגנון המאמר."""
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CLAHE(p=0.5),
        A.Resize(img_size, img_size),
    ])


def build_eval_transform(img_size: int = 224):
    """ולידציה/טסט – רק Resize (נורמליזציה נעשית ידנית אחרי זה)."""
    return A.Compose([
        A.Resize(img_size, img_size),
    ])


class GreenSpaceDataset(Dataset):
    def __init__(self, files: List[str], labels: List[int], transform=None):
        self.files = list(files)
        self.labels = list(labels)
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx: int):
        path = self.files[idx]
        label = self.labels[idx]

        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            raise ValueError(f"Cannot read image: {path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            augmented = self.transform(image=img)
            img = augmented["image"]

        # [H,W,C], uint8 -> float32 [0,1]
        img = img.astype(np.float32) / 255.0
        # נורמליזציה לפי ImageNet
        img = (img - IMAGENET_MEAN) / IMAGENET_STD
        # HWC -> CHW
        img = np.transpose(img, (2, 0, 1))  # (C,H,W)

        img_tensor = torch.from_numpy(img).float()          # float32
        label_tensor = torch.tensor(label, dtype=torch.long)
        return img_tensor, label_tensor


def make_filelist(root_dir: Path, class_names: List[str]) -> Tuple[List[str], List[int]]:
    files: List[str] = []
    labels: List[int] = []

    for idx, cname in enumerate(class_names):
        cdir = root_dir / cname
        if not cdir.exists():
            raise FileNotFoundError(f"Missing class folder: {cdir}")
        exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff", "*.webp")
        for e in exts:
            for p in cdir.glob(e):
                files.append(str(p))
                labels.append(idx)

    return files, labels


def stratified_split(files: List[str], labels: List[int], seed: int = 42):
    """
    חלוקה 70/10/20 ל-Train/Val/Test עם שמירה על פרופורציות המחלקות (Stratified).
    """
    files_arr = np.array(files)
    labels_arr = np.array(labels)

    # Train 70%, Temp 30%
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.30, random_state=seed)
    train_idx, temp_idx = next(sss1.split(files_arr, labels_arr))

    X_train, y_train = files_arr[train_idx], labels_arr[train_idx]
    X_temp,  y_temp  = files_arr[temp_idx],  labels_arr[temp_idx]

    # מתוך ה-30%: שליש ל-Validation (10% מהכול), שני שליש ל-Test (20% מהכול)
    val_size = 1.0 / 3.0
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=(1.0 - val_size), random_state=seed)
    val_idx, test_idx = next(sss2.split(X_temp, y_temp))

    X_val,  y_val  = X_temp[val_idx],  y_temp[val_idx]
    X_test, y_test = X_temp[test_idx], y_temp[test_idx]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

# -----------------------------
# בניית מודל – ViT בלבד
# -----------------------------

def build_vit_model(num_classes: int = NUM_CLASSES, pretrained: bool = True) -> nn.Module:
    """
    ViT-B/16 מ-torchvision, עם:
    - משקלי ImageNet
    - החלפת הראש ל-classifier ל-NUM_CLASSES
    - הקפאת כל ה-encoder, אימון רק של הראש (heads)
    """
    model = models.vit_b_16(
        weights=models.ViT_B_16_Weights.IMAGENET1K_V1 if pretrained else None
    )

    # הראש המקורי: model.heads.head (Linear ל-1000)
    in_features = model.heads.head.in_features
    model.heads.head = nn.Linear(in_features, num_classes)

    # הקפאת כל הפרמטרים
    for param in model.parameters():
        param.requires_grad = False

    # שחרור הראש בלבד ללמידה
    for param in model.heads.parameters():
        param.requires_grad = True

    return model

# -----------------------------
# Training / Evaluation helpers
# -----------------------------

def train_one_epoch(model, loader, optimizer, device, criterion):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    pbar = tqdm(loader, desc="Train", leave=False)

    for xb, yb in pbar:
        xb = xb.to(device=device, dtype=torch.float32)
        yb = yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)
        preds = torch.argmax(logits, dim=1)

        all_preds.append(preds.detach().cpu().numpy())
        all_labels.append(yb.detach().cpu().numpy())

        # עדכון מיני-סטטוס על ה-Progress Bar
        seen_samples = len(all_labels) * loader.batch_size
        current_loss = running_loss / max(1, seen_samples)
        pbar.set_postfix(loss=f"{current_loss:.4f}")

    epoch_loss = running_loss / len(loader.dataset)
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, acc


@torch.no_grad()
def eval_epoch(model, loader, device, criterion, desc: str = "Eval"):
    """
    הרצת ולידציה/טסט: מחזיר:
    - loss ממוצע
    - accuracy
    - y_true, y_pred, y_proba (לצורך מדדים נוספים)
    """
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []

    pbar = tqdm(loader, desc=desc, leave=False)

    for xb, yb in pbar:
        xb = xb.to(device=device, dtype=torch.float32)
        yb = yb.to(device)

        logits = model(xb)
        loss = criterion(logits, yb)

        running_loss += loss.item() * xb.size(0)

        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)

        all_preds.append(preds.cpu().numpy())
        all_labels.append(yb.cpu().numpy())
        all_probs.append(probs.cpu().numpy())

        seen_samples = len(all_labels) * loader.batch_size
        current_loss = running_loss / max(1, seen_samples)
        pbar.set_postfix(loss=f"{current_loss:.4f}")

    epoch_loss = running_loss / len(loader.dataset)
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    all_probs = np.concatenate(all_probs)
    acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, acc, all_labels, all_preds, all_probs


def compute_metrics(y_true_idx, y_pred_idx, y_proba, class_names):
    """
    חישוב כל המדדים:
    - Accuracy
    - Precision/Recall/F1 Macro
    - Cohen's Kappa
    - ROC-AUC Macro OVR
    - Classification Report + Confusion Matrix
    """
    num_classes = len(class_names)
    y_true_oh = np.eye(num_classes)[y_true_idx]

    acc = accuracy_score(y_true_idx, y_pred_idx)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="macro", zero_division=0
    )
    kappa = cohen_kappa_score(y_true_idx, y_pred_idx)

    try:
        rocauc = roc_auc_score(y_true_oh, y_proba, multi_class="ovr", average="macro")
    except Exception:
        rocauc = float("nan")

    report = classification_report(
        y_true_idx, y_pred_idx, target_names=class_names, digits=4, zero_division=0
    )
    cm = confusion_matrix(y_true_idx, y_pred_idx)

    return {
        "accuracy": acc,
        "precision_macro": prec,
        "recall_macro": rec,
        "f1_macro": f1,
        "kappa": kappa,
        "roc_auc_macro_ovr": rocauc,
        "report": report,
        "confusion_matrix": cm.tolist(),
    }

# -----------------------------
# main – כל הפייפליין ל-ViT
# -----------------------------

def main():
    global args

    # reproducibility
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    random.seed(args.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    root = Path(args.root_dir)
    assert root.exists(), f"Root dir not found: {root}"

    os.makedirs(args.save_dir, exist_ok=True)

    # 1) איסוף קבצים
    files, labels = make_filelist(root, CLASS_NAMES)
    print(f"נמצאו {len(files)} קבצים.")
    uniq, counts = np.unique(labels, return_counts=True)
    for i, c in zip(uniq, counts):
        print(f"  {CLASS_NAMES[i]}: {c}")

    # 2) חלוקה 70/10/20
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = stratified_split(
        files, labels, seed=args.seed
    )
    print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

    # 3) DataLoaders – num_workers=0 ל-Notebook/Windows
    train_tf = build_train_transform(IMG_SIZE)
    eval_tf  = build_eval_transform(IMG_SIZE)

    train_ds = GreenSpaceDataset(X_train, y_train, transform=train_tf)
    val_ds   = GreenSpaceDataset(X_val,   y_val,   transform=eval_tf)
    test_ds  = GreenSpaceDataset(X_test,  y_test,  transform=eval_tf)

    train_loader = DataLoader(
        train_ds, batch_size=args.batch, shuffle=True,
        num_workers=0, pin_memory=True
    )
    val_loader = DataLoader(
        val_ds, batch_size=args.batch, shuffle=False,
        num_workers=0, pin_memory=True
    )
    test_loader = DataLoader(
        test_ds, batch_size=args.batch, shuffle=False,
        num_workers=0, pin_memory=True
    )

    # 4) ViT + Optimizer + Scheduler
    model = build_vit_model(NUM_CLASSES, pretrained=True)
    model = model.to(device).float()

    # רק הפרמטרים שניתנים ללמידה (הראש)
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    print(f"Trainable parameters (ViT head only): {len(trainable_params)} tensors")

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(trainable_params, lr=1e-3, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", factor=0.1, patience=2
    )

    # 5) לולאת אימון + מדדים ב-Validation
    best_val_acc = 0.0
    best_state = None
    epochs_no_improve = 0

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
        "val_f1_macro": [],
        "val_kappa": [],
        "val_roc_auc": [],
    }

    for epoch in range(1, args.epochs + 1):
        print(f"\nEpoch {epoch}/{args.epochs}  (arch={args.arch})")

        train_loss, train_acc = train_one_epoch(
            model, train_loader, optimizer, device, criterion
        )

        val_loss, val_acc, y_val_true, y_val_pred, y_val_proba = eval_epoch(
            model, val_loader, device, criterion, desc="Val"
        )

        # מדדים מלאים על ה-Validation
        val_metrics = compute_metrics(y_val_true, y_val_pred, y_val_proba, CLASS_NAMES)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)
        history["val_f1_macro"].append(val_metrics["f1_macro"])
        history["val_kappa"].append(val_metrics["kappa"])
        history["val_roc_auc"].append(val_metrics["roc_auc_macro_ovr"])

        print(
            f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}"
        )
        print(
            f"  Val   Loss: {val_loss:.4f} | "
            f"Val Acc: {val_acc:.4f} | "
            f"Val F1_macro: {val_metrics['f1_macro']:.4f} | "
            f"Val Kappa: {val_metrics['kappa']:.4f} | "
            f"Val ROC-AUC: {val_metrics['roc_auc_macro_ovr']:.4f}"
        )

        # Early Stopping על בסיס Accuracy (כמו ResNet, כדי להישאר קרוב למאמר)
        scheduler.step(val_acc)

        if val_acc > best_val_acc + 1e-4:
            best_val_acc = val_acc
            best_state = model.state_dict()
            epochs_no_improve = 0
            print("  >>> Improvement on Val (Accuracy), saving best model in memory.")
        else:
            epochs_no_improve += 1
            print(f"  No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= args.patience:
            print(f"Early stopping (patience={args.patience})")
            break

    # טוענים את המודל הטוב ביותר
    if best_state is not None:
        model.load_state_dict(best_state)

    # 6) הערכה על Test + מדדים מלאים
    test_loss, test_acc, y_true, y_pred, y_proba = eval_epoch(
        model, test_loader, device, criterion, desc="Test"
    )
    print(f"\n=== Evaluation on Test Set (ViT) ===")
    print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

    test_metrics = compute_metrics(y_true, y_pred, y_proba, CLASS_NAMES)
    print("\nMetrics (summary – Test):")
    for k in [
        "accuracy",
        "precision_macro",
        "recall_macro",
        "f1_macro",
        "kappa",
        "roc_auc_macro_ovr",
    ]:
        v = test_metrics[k]
        if isinstance(v, float):
            print(f"{k}: {v:.4f}")
        else:
            print(f"{k}: {v}")

    print("\nClassification Report (Test):\n", test_metrics["report"])
    print("\nConfusion Matrix (Test):\n", np.array(test_metrics["confusion_matrix"]))

    # 7) שמירת מודל, מדדי Test והיסטוריית אימון
    model_path = os.path.join(args.save_dir, f"model_{args.arch}.pt")
    torch.save(model.state_dict(), model_path)
    print(f"\nSaved ViT model to: {model_path}")

    out_metrics_path = os.path.join(args.save_dir, "test_metrics.json")
    with open(out_metrics_path, "w", encoding="utf-8") as f:
        json.dump(test_metrics, f, ensure_ascii=False, indent=2)
    print(f"Saved test metrics to: {out_metrics_path}")

    hist_path = os.path.join(args.save_dir, "history.json")
    with open(hist_path, "w", encoding="utf-8") as f:
        json.dump(history, f, ensure_ascii=False, indent=2)
    print(f"Saved training history to: {hist_path}")

    # 8) שמירת מידע על החלוקה (Train/Val/Test)
    split_info = {
        "class_names": CLASS_NAMES,
        "train_files": X_train.tolist(),
        "val_files": X_val.tolist(),
        "test_files": X_test.tolist(),
        "arch": args.arch,
        "img_size": IMG_SIZE,
        "batch": args.batch,
        "epochs": args.epochs,
        "seed": args.seed,
    }
    split_path = os.path.join(args.save_dir, "split_config.json")
    with open(split_path, "w", encoding="utf-8") as f:
        json.dump(split_info, f, ensure_ascii=False, indent=2)
    print(f"\nSaved split config to: {split_path}")

    print("\nAll done (ViT-only pipeline).")


# להריץ ב-Notebook:
main()


Using device: cuda
נמצאו 987 קבצים.
  Healthy: 334
  Dried: 322
  Contaminated: 331
Train: 690 | Val: 98 | Test: 199
Trainable parameters (ViT head only): 2 tensors

Epoch 1/50  (arch=vit_b_16)


Train:   0%|          | 0/22 [00:00<?, ?it/s]

Val:   0%|          | 0/4 [00:00<?, ?it/s]

  Train Loss: 0.6887 | Train Acc: 0.7391
  Val   Loss: 0.3793 | Val Acc: 0.8776 | Val F1_macro: 0.8772 | Val Kappa: 0.8163 | Val ROC-AUC: 0.9706
  >>> Improvement on Val (Accuracy), saving best model in memory.

Epoch 2/50  (arch=vit_b_16)


Train:   0%|          | 0/22 [00:00<?, ?it/s]

...

Train:   0%|          | 0/22 [00:00<?, ?it/s]

Val:   0%|          | 0/4 [00:00<?, ?it/s]

  Train Loss: 0.0401 | Train Acc: 0.9913
  Val   Loss: 0.0629 | Val Acc: 0.9898 | Val F1_macro: 0.9897 | Val Kappa: 0.9847 | Val ROC-AUC: 0.9997
  No improvement for 7 epoch(s).
Early stopping (patience=7)


Test:   0%|          | 0/7 [00:00<?, ?it/s]


=== Evaluation on Test Set (ViT) ===
Test Loss: 0.0697 | Test Acc: 0.9749

Metrics (summary – Test):
accuracy: 0.9749
precision_macro: 0.9755
recall_macro: 0.9751
f1_macro: 0.9748
kappa: 0.9623
roc_auc_macro_ovr: 0.9992

Classification Report (Test):
               precision    recall  f1-score   support

     Healthy     0.9844    0.9403    0.9618        67
       Dried     0.9420    1.0000    0.9701        65
Contaminated     1.0000    0.9851    0.9925        67

    accuracy                         0.9749       199
   macro avg     0.9755    0.9751    0.9748       199
weighted avg     0.9758    0.9749    0.9749       199


Confusion Matrix (Test):
 [[63  4  0]
 [ 0 65  0]
 [ 1  0 66]]

Saved ViT model to: outputs_gsq_vit\model_vit_b_16.pt
Saved test metrics to: outputs_gsq_vit\test_metrics.json
Saved training history to: outputs_gsq_vit\history.json

Saved split config to: outputs_gsq_vit\split_config.json

All done (ViT-only pipeline).
